In [131]:
from fin_data_fundamentals import find_fundamentals
from fin_data_fundamentals import get_fundamentals
from alpha_vantage.foreignexchange import ForeignExchange
from alpha_vantage.techindicators import TechIndicators
from alpha_vantage.timeseries import TimeSeries
from decouple import config
import pandas as pd
import numpy as np
import quandl
import datetime
from sklearn.preprocessing import MinMaxScaler

In [132]:
from fin_data import DailyTimeSeries

In [133]:
from keras.preprocessing.sequence import TimeseriesGenerator

In [134]:
tesla = DailyTimeSeries('TSLA')

df = tesla.initiate()

################################################################### 
 Ticker:  TSLA 
 Last Refreshed:  2019-09-03 
 Data Retrieved:  Daily Prices (open, high, low, close) and Volumes 
 ###################################################################


In [135]:
df.head()

,TSLA open,TSLA high,TSLA low,TSLA close,TSLA volume
date,,,,,
2010-06-29,19.00,24.9984,17.54,23.89,18766300.0
2010-06-30,25.79,30.4190,23.30,23.83,17187100.0
2010-07-01,25.00,25.9200,20.27,21.96,8218800.0
2010-07-02,23.00,23.0998,18.71,19.20,5139800.0
2010-07-06,20.00,20.0000,15.83,16.11,6866900.0


In [136]:

macro_list = ["housing_index", "trade_index", "confidence_index", "longterm_rates"]

In [137]:
df = tesla.add_macro(df, macro_list)

################################################################### 
 Index: Nominal Home Price Index Added 
 ###################################################################


C:\Users\Billi\repos\market_makers\Data-Science\data\fin_data.py:293: UserWarning: The latest value available for Housing Index is from January 2019.
  warnings.warn("The latest value available for Housing Index is from January 2019.")


################################################################### 
 Trade Weighted U.S. Dollar Index: Broad Added 
 ###################################################################
################################################################### 
 Index: Yale Investor Behavior Project Added 
 ###################################################################
################################################################### 
 US Treasury Bond Long-Term Rates Added 
 ###################################################################


In [138]:
fund_list = ["operatingrevenue", "totalrevenue", "netincome", "totaloperatingexpenses", "totalgrossprofit", "totaloperatingincome"]

In [139]:
df = tesla.add_fundamentals(df, fund_list)

################################################################### 
 Ticker:  TSLA 
 Fundamentals Retrieved:  ['TSLA open' 'TSLA high' 'TSLA low' 'TSLA close' 'TSLA volume'
 'housing_index' 'trade_value' 'conf_index' 'conf_index_SE' '10 Yrs Rates'
 '20-Yr Maturity Rate' 'TSLA_operatingrevenue' 'TSLA_totalrevenue'
 'TSLA_netincome' 'TSLA_totaloperatingexpenses' 'TSLA_totalgrossprofit'
 'TSLA_totaloperatingincome'] 
 ###################################################################
################################################################### 
 Ticker:  TSLA 
 Retrieved Data Start Date:  2011-11-14 
 Retrieved Data End Date:  2019-07-29 
 Data Retrieved:  ['TSLA_operatingrevenue', 'TSLA_totalrevenue', 'TSLA_netincome', 'TSLA_totaloperatingexpenses', 'TSLA_totalgrossprofit', 'TSLA_totaloperatingincome'] 
 ###################################################################


In [140]:
df.head()


,TSLA open,TSLA high,TSLA low,TSLA close,TSLA volume,housing_index,trade_value,conf_index,conf_index_SE,10 Yrs Rates,20-Yr Maturity Rate,TSLA_operatingrevenue,TSLA_totalrevenue,TSLA_netincome,TSLA_totaloperatingexpenses,TSLA_totalgrossprofit,TSLA_totaloperatingincome
date,,,,,,,,,,,,,,,,,
2010-06-29,19.00,24.9984,17.54,23.89,18766300.0,147.031,104.0133,65.43,3.47,3.63,3.76,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0
2010-06-30,25.79,30.4190,23.30,23.83,17187100.0,147.695,104.4167,65.64,3.40,3.61,3.74,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0
2010-07-01,25.00,25.9200,20.27,21.96,8218800.0,147.695,104.4167,65.64,3.40,3.58,3.71,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0
2010-07-02,23.00,23.0998,18.71,19.20,5139800.0,147.695,104.4167,65.64,3.40,3.64,3.77,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0
2010-07-06,20.00,20.0000,15.83,16.11,6866900.0,147.695,104.4167,65.64,3.40,3.58,3.71,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0


In [141]:
df.shape

(2163, 17)

### Target Feature Engineering ###

- For now, we are using percent change as the target feature. In the future, we would be engineering more features that possibly encompass the data more accurately


In [142]:
df['percent_change'] = ((df['TSLA close'].shift(-1) - df['TSLA close']) / (df['TSLA close']))*100


df['percent_change'].head()

date
2010-06-29    -0.251151
2010-06-30    -7.847251
2010-07-01   -12.568306
2010-07-02   -16.093750
2010-07-06    -1.924271
Name: percent_change, dtype: float64

In [143]:
df.head()

,TSLA open,TSLA high,TSLA low,TSLA close,TSLA volume,housing_index,trade_value,conf_index,conf_index_SE,10 Yrs Rates,20-Yr Maturity Rate,TSLA_operatingrevenue,TSLA_totalrevenue,TSLA_netincome,TSLA_totaloperatingexpenses,TSLA_totalgrossprofit,TSLA_totaloperatingincome,percent_change
date,,,,,,,,,,,,,,,,,,
2010-06-29,19.00,24.9984,17.54,23.89,18766300.0,147.031,104.0133,65.43,3.47,3.63,3.76,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0,-0.251151
2010-06-30,25.79,30.4190,23.30,23.83,17187100.0,147.695,104.4167,65.64,3.40,3.61,3.74,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0,-7.847251
2010-07-01,25.00,25.9200,20.27,21.96,8218800.0,147.695,104.4167,65.64,3.40,3.58,3.71,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0,-12.568306
2010-07-02,23.00,23.0998,18.71,19.20,5139800.0,147.695,104.4167,65.64,3.40,3.64,3.77,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0,-16.093750
2010-07-06,20.00,20.0000,15.83,16.11,6866900.0,147.695,104.4167,65.64,3.40,3.58,3.71,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0,-1.924271


In [144]:
# For the last couple of months, tesla had a lot of missing fundamentals data, so we have th drop the null values.

print(df.isnull().sum())
print(df.shape)

df = df.dropna()

df.shape
# loses about 60 observations.

TSLA open                       0
TSLA high                       0
TSLA low                        0
TSLA close                      0
TSLA volume                     0
housing_index                   0
trade_value                     0
conf_index                      0
conf_index_SE                   0
10 Yrs Rates                    0
20-Yr Maturity Rate             0
TSLA_operatingrevenue          60
TSLA_totalrevenue              60
TSLA_netincome                 60
TSLA_totaloperatingexpenses    60
TSLA_totalgrossprofit          60
TSLA_totaloperatingincome      60
percent_change                  1
dtype: int64
(2163, 18)


(2103, 18)

### Multiple Features

- Let's try to have multiple features in the notebook to see if that helps the model.
- Time Series Forecasting As Supervised Learning:
- reframing the time sereis allows the access to standard linear and nonlinear machine learning algorithms
- Supervised learning: algorithm learning from the training dataset. Has specific features and targets with values.
- We have to make a sliding window i.e. taking in the previous time steps as input variables, we use the next time step as the output variable.
- Also knowns as sliding window method, window method or a lag method. 
- How the sliding window work would be we would have all 18 features to predict the percent change move. 
###### Multi-Step Forecasting:
- One-Step Forecast: where the next day's percent change is predicted
- Multi-Step Forecase: Two or more future time steps are to be predicted.


##### Pandas shift() Function#####

- in order to create lag observations to forecast observations for a time series dataset.
- Typically, current time(t) and future times (t+1, t+n) are forecast times and past observations (t-1, t-n) are used to make forecasts.


In [145]:
import math
#from keras.preprocessing.sequence import TimeseriesGenerator

In [146]:
df.head()

,TSLA open,TSLA high,TSLA low,TSLA close,TSLA volume,housing_index,trade_value,conf_index,conf_index_SE,10 Yrs Rates,20-Yr Maturity Rate,TSLA_operatingrevenue,TSLA_totalrevenue,TSLA_netincome,TSLA_totaloperatingexpenses,TSLA_totalgrossprofit,TSLA_totaloperatingincome,percent_change
date,,,,,,,,,,,,,,,,,,
2010-06-29,19.00,24.9984,17.54,23.89,18766300.0,147.031,104.0133,65.43,3.47,3.63,3.76,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0,-0.251151
2010-06-30,25.79,30.4190,23.30,23.83,17187100.0,147.695,104.4167,65.64,3.40,3.61,3.74,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0,-7.847251
2010-07-01,25.00,25.9200,20.27,21.96,8218800.0,147.695,104.4167,65.64,3.40,3.58,3.71,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0,-12.568306
2010-07-02,23.00,23.0998,18.71,19.20,5139800.0,147.695,104.4167,65.64,3.40,3.64,3.77,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0,-16.093750
2010-07-06,20.00,20.0000,15.83,16.11,6866900.0,147.695,104.4167,65.64,3.40,3.58,3.71,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0,-1.924271


In [147]:
# if we want to use the day change value as the time-series data instead we would have to convert
# the dataset into day change values by this:

def delta_time_series(data):
    return data[1:] - data[:-1]


In [148]:
df.head()

,TSLA open,TSLA high,TSLA low,TSLA close,TSLA volume,housing_index,trade_value,conf_index,conf_index_SE,10 Yrs Rates,20-Yr Maturity Rate,TSLA_operatingrevenue,TSLA_totalrevenue,TSLA_netincome,TSLA_totaloperatingexpenses,TSLA_totalgrossprofit,TSLA_totaloperatingincome,percent_change
date,,,,,,,,,,,,,,,,,,
2010-06-29,19.00,24.9984,17.54,23.89,18766300.0,147.031,104.0133,65.43,3.47,3.63,3.76,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0,-0.251151
2010-06-30,25.79,30.4190,23.30,23.83,17187100.0,147.695,104.4167,65.64,3.40,3.61,3.74,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0,-7.847251
2010-07-01,25.00,25.9200,20.27,21.96,8218800.0,147.695,104.4167,65.64,3.40,3.58,3.71,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0,-12.568306
2010-07-02,23.00,23.0998,18.71,19.20,5139800.0,147.695,104.4167,65.64,3.40,3.64,3.77,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0,-16.093750
2010-07-06,20.00,20.0000,15.83,16.11,6866900.0,147.695,104.4167,65.64,3.40,3.58,3.71,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0,-1.924271


#### Normalize the dataset:

In [149]:
df.head()

,TSLA open,TSLA high,TSLA low,TSLA close,TSLA volume,housing_index,trade_value,conf_index,conf_index_SE,10 Yrs Rates,20-Yr Maturity Rate,TSLA_operatingrevenue,TSLA_totalrevenue,TSLA_netincome,TSLA_totaloperatingexpenses,TSLA_totalgrossprofit,TSLA_totaloperatingincome,percent_change
date,,,,,,,,,,,,,,,,,,
2010-06-29,19.00,24.9984,17.54,23.89,18766300.0,147.031,104.0133,65.43,3.47,3.63,3.76,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0,-0.251151
2010-06-30,25.79,30.4190,23.30,23.83,17187100.0,147.695,104.4167,65.64,3.40,3.61,3.74,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0,-7.847251
2010-07-01,25.00,25.9200,20.27,21.96,8218800.0,147.695,104.4167,65.64,3.40,3.58,3.71,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0,-12.568306
2010-07-02,23.00,23.0998,18.71,19.20,5139800.0,147.695,104.4167,65.64,3.40,3.64,3.77,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0,-16.093750
2010-07-06,20.00,20.0000,15.83,16.11,6866900.0,147.695,104.4167,65.64,3.40,3.58,3.71,31241000.0,31241000.0,-34935000.0,47130000.0,9296000.0,-37834000.0,-1.924271


In [150]:
dataset = df.values

In [151]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit_transform(dataset.reshape(-1,1))

array([[0.18358664],
       [0.18358664],
       [0.18358664],
       ...,
       [0.30644219],
       [0.03711406],
       [0.18358664]])

#### Split the dataset into train and test sets

In [152]:
dataset.shape

(2103, 18)

In [154]:
scaled = scaler.transform(df)

In [155]:
train = scaled[:1500, :]
test = scaled[1500:, :]

In [156]:
train.shape

(1500, 18)

In [157]:
X_train, X_test = train[:1500,:17], test[:,:17] 

In [158]:
X_train.shape, X_test.shape

((1500, 17), (603, 17))

In [159]:
y_train, y_test = train[:1500, 17:18], test[:, 17:18]

In [160]:
y_train.shape, y_test.shape

((1500, 1), (603, 1))

In [161]:
look_back = 7 # 7 dats

### TimeseriesGenerator:
Data: takes in data argument i.e. X_train, should be 2D, axis 0 is the time dimension and has timesteps.
target: target for the model i.e. y_train
length: lenght of output sequences (in number of timesteps)
sampling_rate: period between successive timesteps within sequences.
stride: sequence between successive output sequences.
batch_size: number of timeseries samples in each batch, i.e. number of days you want to predict the target for.
can also use:
start_index: data points earlier than start_index would not be used in output sequence, this is to split the data for test or validation.
end_index: data points later than end_index would not be used in the output sequences, useful to reserve the data for test or validation.

In [162]:
train_data_generator = TimeseriesGenerator(X_train, y_train, 
                                           length=look_back,
                                           sampling_rate=1, 
                                           stride=1, 
                                           batch_size=3)


In [163]:
test_data_generator = TimeseriesGenerator(X_test, y_test, 
                                          length=look_back, 
                                          sampling_rate=1,
                                          stride=1,
                                          batch_size =1)

### Let's Define the Model:

In [164]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [165]:
model = Sequential()
model.add(LSTM(32, input_shape=(look_back, X_train.shape[1])))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [166]:
history = model.fit_generator(train_data_generator, epochs = 200)

Epoch 1/200
498/498 [==============================] - 6s 12ms/step - loss: 2.8911e-04
Epoch 2/200
498/498 [==============================] - 4s 7ms/step - loss: 1.0696e-05
Epoch 3/200
498/498 [==============================] - 4s 7ms/step - loss: 5.5131e-06
Epoch 4/200
498/498 [==============================] - 3s 7ms/step - loss: 4.0988e-06
Epoch 5/200
498/498 [==============================] - 4s 8ms/step - loss: 3.6098e-06
Epoch 6/200
498/498 [==============================] - 3s 7ms/step - loss: 4.4898e-06
Epoch 7/200
498/498 [==============================] - 4s 7ms/step - loss: 2.7241e-06
Epoch 8/200
498/498 [==============================] - 4s 8ms/step - loss: 4.4191e-06
Epoch 9/200
498/498 [==============================] - 3s 6ms/step - loss: 8.1269e-06
Epoch 10/200
498/498 [==============================] - 4s 8ms/step - loss: 1.9840e-06
Epoch 11/200
498/498 [==============================] - 4s 7ms/step - loss: 7.1694e-06
Epoch 12/200
498/498 [=============================

In [167]:
history

In [169]:
#!pip install pydot

## To see what your model looks:

from keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True)
from IPython.display import Image
Image(filename='model.png')

In [170]:
# history = model.fit_generator(train_data_generator, epochs=100).history

In [171]:
model.save('first_model.h')

In [172]:
from keras.models import load_model

model = load_model('first_model.h')

In [173]:
model.evaluate_generator(test_data_generator)

1.5216502205986937e-06

In [174]:
trainPredict = model.predict_generator(train_data_generator)
trainPredict.shape

(1493, 1)

In [175]:
testPredict = model.predict_generator(test_data_generator)
testPredict.shape

(596, 1)

In [176]:
## scale values back to real index/price range.

trainPredict = scaler.inverse_transform(trainPredict)
testPredict = scaler.inverse_transform(testPredict)
dataset_y = scaler.inverse_transform(dataset_y)


NameError: name 'dataset_y' is not defined

In [ ]:
from math import mean_squared_error

In [ ]:
# calculating the room mean squared erro

train_score = math.sqrt(mean_squared_error(y_train[:,0], trainPredict[:,0]))
print('Train Score: %0.2f RMSE' % (train_score))

test_score = math.sqrt(mean_squared_error(y_test[:, 0], testPredict[:, 0]))
print('Test Score: %.2f RMSE' % (test_score))

# this would print out the RMSE score

In [ ]:
# you can also plot the train and test predictions.


### ANOTHER MODEL:

In [ ]:
regressor = Sequential()


regressor.add(LSTM(32, input_shape=(look_back, X_train.shape[1])))
regressor.add(Dropout(0.2))
# 32 is the batch_size

# Adding the second layer

regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))

# Adding a third layer
regressor.add(LSTM(units=50, return_sequences=False))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units=1))

# Compiling the RNN
regressor.compile(optimizer='adam', loss='mean_squared_error')

# Fitting the RNN to the Training set

regressor.fit(X_train, y_train, epochs =100, batch_size =32)
